In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from preprocessing.comment_process import normalize_blanklines, strip_cryptol_comments_all

In [2]:
jsonl_path = "data/training_datasets/verified_nomods.jsonl"

# --- load dataset ---
rows = []
with open(jsonl_path, "r", encoding="utf-8") as f:
    for line in f:
        rows.append(json.loads(line))

print(f"Loaded {len(rows)} rows")

nomod_df = pd.DataFrame(rows)

Loaded 865 rows


In [3]:
nomod_df.head()

,filename,filetype,content
0,cryptol/examples/splitAt.cry,cry,"x = [1,2,3,4] : [_][8]\n\ny = (splitAt x) : ([..."
1,cryptol/examples/AE.cry,cry,// WORK IN PROGRESS\n\n/*\nImplementation of t...
2,cryptol/examples/xor_cipher.cry,cry,encrypt : {a}(fin a) => [8] -> [a][8] -> [a][8...
3,cryptol/examples/zero_weird.cry,cry,x : {a}() => a -> [16]\nx v = zero v \n\nprope...
4,cryptol/examples/builtin_lifting.cry,cry,"//builtins lift over tuples, seqs, and records..."


In [4]:
nocomment_df = nomod_df.copy()
nocomment_df["content"] = nocomment_df["content"].apply(strip_cryptol_comments_all)


In [5]:
nocomment_df.head()

,filename,filetype,content
0,cryptol/examples/splitAt.cry,cry,"x = [1,2,3,4] : [_][8]\n\ny = (splitAt x) : ([..."
1,cryptol/examples/AE.cry,cry,\n\nmodule AE where\n\nparameter\n type A : *...
2,cryptol/examples/xor_cipher.cry,cry,encrypt : {a}(fin a) => [8] -> [a][8] -> [a][8...
3,cryptol/examples/zero_weird.cry,cry,x : {a}() => a -> [16]\nx v = zero v \n\nprope...
4,cryptol/examples/builtin_lifting.cry,cry,"\n\nx = [True,False]\ny = [False,True]\n\nprop..."


In [6]:
nocomment_df["content"] = nocomment_df["content"].apply(normalize_blanklines)

In [7]:
nocomment_df["variant"] = "without_comments"
nocomment_df.head()

,filename,filetype,content,variant
0,cryptol/examples/splitAt.cry,cry,"x = [1,2,3,4] : [_][8]\n\ny = (splitAt x) : ([...",without_comments
1,cryptol/examples/AE.cry,cry,module AE where\n\nparameter\n type A : * ...,without_comments
2,cryptol/examples/xor_cipher.cry,cry,encrypt : {a}(fin a) => [8] -> [a][8] -> [a][8...,without_comments
3,cryptol/examples/zero_weird.cry,cry,x : {a}() => a -> [16]\nx v = zero v \n\nprope...,without_comments
4,cryptol/examples/builtin_lifting.cry,cry,"x = [True,False]\ny = [False,True]\n\nproperty...",without_comments


In [8]:
nocomment_df.to_json("data/training_datasets/verified_nocomments.jsonl", orient="records", lines=True, force_ascii=False)


In [9]:
from preprocessing.comment_extractor import extract_strip_cry_comments
# Create Hybrid Dataset
DECISION_CACHE_PATH = "GPT_comment_decisions_cache.jsonl"
comment_rows = []
dataset_rows = []
for index, row in nomod_df.iterrows():

    if row.filetype == "cry":
        comments, file_record_ = extract_strip_cry_comments(
            filename=row.filename,
            content=row.content,
            llm_model_name="gpt-oss:20b",
            decision_cache_path=DECISION_CACHE_PATH
        )
        comment_rows.extend(comments)
        file_record = {
            "filename": file_record_["filename"],
            "filetype": row.filetype,
            "content": file_record_["content"],
            "variant": "hybrid"
        }
    else:
        file_record = {
            "filename": row.filename,
            "filetype": row.filetype,
            "content": row.content,
            "variant": "hybrid"
        }
    dataset_rows.append(file_record)

comment_df = pd.DataFrame(comment_rows)
hybrid_df = pd.DataFrame(dataset_rows)



In [10]:
comment_df.head()

,filename,sha1,comment,keep,snippet
0,cryptol/examples/AE.cry,5e86684a393aa148fd0a454958fa967d95065be8,// WORK IN PROGRESS,False,
1,cryptol/examples/AE.cry,259850075ed51d4fd666df2fe0a2063d600374ce,/*\nImplementation of the algorithms from the ...,True,module AE where\n\nparameter\n type A : * ...
2,cryptol/examples/AE.cry,150ca06f0e9d2946cbce4dc93eae2277e1402f91,// State type,False,type K : *
3,cryptol/examples/AE.cry,84c8479e2cda530b361b5d04e7c090204713d38e,// Key type,False,type n : #
4,cryptol/examples/AE.cry,d552c54a82a17f171563b8dd15d500d5212539b2,// Block size,False,type p : #


In [11]:
hybrid_df.head()

,filename,filetype,content,variant
0,cryptol/examples/splitAt.cry,cry,"x = [1,2,3,4] : [_][8]\n\ny = (splitAt x) : ([...",hybrid
1,cryptol/examples/AE.cry,cry,/*\nImplementation of the algorithms from the ...,hybrid
2,cryptol/examples/xor_cipher.cry,cry,encrypt : {a}(fin a) => [8] -> [a][8] -> [a][8...,hybrid
3,cryptol/examples/zero_weird.cry,cry,x : {a}() => a -> [16]\nx v = zero v \n\nprope...,hybrid
4,cryptol/examples/builtin_lifting.cry,cry,"//builtins lift over tuples, seqs, and records...",hybrid


In [12]:
HYBRID_PATH = "data/training_datasets/GPTverified_hybrid.jsonl"
#NOCOMMENT_PATH = "data/training_datasets/nocomments.jsonl"
COMMENT_STAT_PATH = "data/GPTcomment_stats.jsonl"

hybrid_df.to_json(HYBRID_PATH, lines=True, orient="records")
#nocomment_df.to_json(NOCOMMENT_PATH, lines=True, orient="records")
comment_df.to_json(COMMENT_STAT_PATH, lines=True, orient="records")

